In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 29 15:39:51 2023

This script trains PINN model for TDSF supplied by AFTAC-CRADA
Inputs: yield (w), depth (h), time (t)
Output: displacement (S)

@author: shhong
"""

#%% Load Moduels

import os
root = os.path.abspath('')
# root = '/home/ubuntu/tdsf'

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
import matplotlib.pyplot as plt

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer
from sklearn.preprocessing import MinMaxScaler

# from sklearn.metrics import confusion_matrix
# from sklearn.preprocessing import OneHotEncoder

# device = torch.device('cuda:0')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

study = 'noise_study_ood'
#%% Load Data

# path = 'E:\Hong\Research Projects\PINN\Data'
path = f'{root}/data'

filename = 'tdsf_002_train.pickle'
df_train_all = pd.read_pickle(path + '/' + filename)

filename = 'tdsf_002_val.pickle'
df_test_all = pd.read_pickle(path + '/' + filename)

df_data_all = pd.concat([df_train_all, df_test_all], axis=0)


#%% Select Material

# [Tuff-Rhyolite, Granite, Shale, Salt, Wet-Granite, Wet-Tuff]

name_prefix = '_FAR.'
name_material = 'Tuff-Rhyolite'

df_data = df_data_all[df_data_all['material'] == name_prefix + name_material]
df_data = df_data.reset_index(drop=True)

# splits df into train and test by yield and depth quantiles
qnts = [.1, .9]
ylds, dpths = df_data.YIELD.quantile(qnts, interpolation='nearest'), df_data.DEPTH.quantile(qnts, interpolation='nearest')
train_qnt = (df_data.YIELD >= ylds.min()) & (df_data.YIELD <= ylds.max()) & (df_data.DEPTH >= dpths.min()) & (df_data.DEPTH <= dpths.max())
df_train = df_data[train_qnt].reset_index(drop=True)
df_test = df_data[~train_qnt].reset_index(drop=True)

# plot train and test yields and depths
plt.figure()
plt.scatter(df_train.YIELD, df_train.DEPTH, label='Train', color='black', alpha=0.2)
plt.scatter(df_test.YIELD, df_test.DEPTH, label='Test', color='red', alpha=0.2)
plt.xlabel('Yield')
plt.ylabel('Depth')
plt.legend()
plt.title('Train and Test domains')
plt.savefig(f'{root}/results/{study}/train_test_domains.png')
plt.close('all')



#%% Data Arrangement

trunc = 1000
ds = 10
dt = 0.001

tv = np.arange(1,trunc+1)*dt
tv = tv[::ds]

# Define Train Set
tdsf = df_train.DATA
wvec = df_train.YIELD
hvec = df_train.DEPTH
t_tdsf = tdsf[0][:trunc]
Xtrain = np.concatenate((wvec[0]*np.ones((tv.size,1)), hvec[0]*np.ones((tv.size,1)), np.reshape(tv,(-1,1))), axis=1)
Ytrain = np.reshape(t_tdsf[::ds], (-1,1))
for ndx in range(1, df_train.shape[0]):
    t_tdsf = tdsf[ndx][:trunc]
    Xtemp = np.concatenate((wvec[ndx]*np.ones((tv.size,1)), hvec[ndx]*np.ones((tv.size,1)), np.reshape(tv,(-1,1))), axis=1)        
    Xtrain = np.concatenate((Xtrain, Xtemp), axis=0)
    Ytrain = np.concatenate((Ytrain, np.reshape(t_tdsf[::ds], (-1,1))), axis=0)
Xtrain = Xtrain.astype(np.float32)
Ytrain = Ytrain.astype(np.float32)

# Define Test Set
tdsf = df_test.DATA
wvec = df_test.YIELD
hvec = df_test.DEPTH
t_tdsf = tdsf[0][:trunc]
Xtest = np.concatenate((wvec[0]*np.ones((tv.size,1)), hvec[0]*np.ones((tv.size,1)), np.reshape(tv,(-1,1))), axis=1)
Ytest = np.reshape(t_tdsf[::ds], (-1,1))
for ndx in range(1, df_test.shape[0]):
    t_tdsf = tdsf[ndx][:trunc]
    Xtemp = np.concatenate((wvec[ndx]*np.ones((tv.size,1)), hvec[ndx]*np.ones((tv.size,1)), np.reshape(tv,(-1,1))), axis=1)        
    Xtest = np.concatenate((Xtest, Xtemp), axis=0)
    Ytest = np.concatenate((Ytest, np.reshape(t_tdsf[::ds], (-1,1))), axis=0)
Xtest = Xtest.astype(np.float32)
Ytest = Ytest.astype(np.float32)


#%% Preprocessing

# Min-Max Normalization
Xscaler = MinMaxScaler()
Xscaler.fit(Xtrain)
MaxX = Xscaler.data_max_
MinX = Xscaler.data_min_

Yscaler = MinMaxScaler()
Yscaler.fit(Ytrain)
MaxY = Yscaler.data_max_
MinY = Yscaler.data_min_

Ztrain = Yscaler.transform(Ytrain)
Ztest = Yscaler.transform(Ytest)

# Shuffle Data
numdata = Xtrain.shape[0]
np.random.seed(0)
randperm = np.random.permutation(numdata)
Utrain = Xtrain[randperm,:]
Ztrain_base = Ztrain[randperm,:]

/tmp/ipykernel_4234/3255806211.py:20: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
noise = [0.00, 0.01, 0.1, 0.2, 0.5]
for sigma in noise:

    path = f'{root}/results/{study}/{sigma}'
    if not os.path.exists(path): os.makedirs(path)
    else:
        print(f'{root}/results/{study}/{sigma} exists, skipping')
        continue

    # Gaussian Noise
    # sigma = 0.00
    mu = 0.00
    np.random.seed(1)
    Ztrain = (Ztrain_base + sigma*np.random.randn(numdata,1) + mu).astype(np.float32)

    numIn = Utrain.shape[1]
    numOut = Ztrain.shape[1]


    #%% Define Classes

    class ANN_Dataset(Dataset):    

        # Initialize your data
        def __init__(self, x, y):        
            self.len = x.shape[0]
            self.x_data = torch.from_numpy(x)
            self.y_data = torch.from_numpy(y)

        def __getitem__(self, index):
            return self.x_data[index], self.y_data[index]

        def __len__(self):
            return self.len


    # Fully Connected NN
    class FCNN_Net(nn.Module):

        def __init__(self, numIn, numOut):
            super(FCNN_Net, self).__init__()
            self.fc1 = nn.Linear(numIn, 24)
            self.fc2 = nn.Linear(24, 6)
            self.fc3 = nn.Linear(6, numOut)
            
        def forward(self, w, h, t):        
            inputs = torch.cat([w,h,t],axis=1) # combined two arrays of 1 columns each to one array of 2 columns        
            # in_size = x.size(0)
            # inputs = x.view(in_size, -1)
            layer1_out = torch.tanh(self.fc1(inputs))
            layer2_out = torch.tanh(self.fc2(layer1_out))
            output = (self.fc3(layer2_out))
            return output


    #%% Defined Functions


    ## PDE as loss function. Thus would use the network which we call as u_theta
    def f_partial(data, net, MaxX, MinX, MaxY, MinY):
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        xmax = torch.from_numpy(MaxX).to(device)
        xmin = torch.from_numpy(MinX).to(device)
        ymax = torch.from_numpy(MaxY).to(device)
        ymin = torch.from_numpy(MinY).to(device)
        
        wvec = Variable(torch.reshape(data[:,0], (-1,1)), requires_grad=True).to(device)
        hvec = Variable(torch.reshape(data[:,1], (-1,1)), requires_grad=True).to(device)
        tvec = Variable(torch.reshape(data[:,2], (-1,1)), requires_grad=True).to(device)
            
        # applying normalized input
        # x = torch.cat([(wvec-xmin[0])/(xmax[0]-xmin[0]),(hvec-xmin[1])/(xmax[1]-xmin[1]),(tvec-xmin[2])/(xmax[2]-xmin[2])],axis=1)    
        s = net((wvec-xmin[0])/(xmax[0]-xmin[0]), (hvec-xmin[1])/(xmax[1]-xmin[1]), (tvec-xmin[2])/(xmax[2]-xmin[2])) # the dependent variable s is given by the network based on independent variables w, h, t

        # remove normalization
        s = s*(ymax-ymin) + ymin
        
        s_w = torch.autograd.grad(s.sum(), wvec, create_graph=True)[0]
        s_h = torch.autograd.grad(s.sum(), hvec, create_graph=True)[0] 
            
        ww = torch.reshape(data[:,0], (-1,1))
        hh = torch.reshape(data[:,1], (-1,1))
        tt = torch.reshape(data[:,2], (-1,1))
        
        # define parameters
        ho = 122
        Ro = 202
        go = 26
        P1 = 3.6*1e6
        P2 = 5.0*1e6
        n = 2.4
        pv = 3500
        sv = 2021
        rho = 2000
        
        dsdR = torch.zeros(ww.shape).to(device)

        Rel = Ro*((ho/hh)**(1/n))*(ww**(1/3))
        ga = go*Ro/Rel

        mu = rho*(sv**2)
        lam = rho*(pv**2)-2*mu    
        wo = pv/Rel
        bet = (lam+2*mu)/(4*mu)
        alp = wo/(2*bet)
        p = wo*(1/2/bet-1/4/bet**2)**(1/2)

        def dF_(tau):
            return (Rel*pv**2)/(4*mu*bet*p)*(-alp*torch.exp(-alp*tau)*torch.sin(p*tau) + p*torch.exp(-alp*tau)*torch.cos(p*tau))

        def dBdR_(tau):
            return (tt-tau)*ga*torch.exp(-ga*(tt-tau))/Rel*P1*(hh/ho) - ((tt-tau)*ga*torch.exp(-ga*(tt-tau))+3*(1-torch.exp(-ga*(tt-tau))))/Rel*P2*((ho/hh)**(1/3))*((Ro/Rel)**3)*(ww**(0.87))

        dtau = 0.001
        tau = torch.tensor(np.arange(0,1,dtau)).to(device).float()
        
        dF = dF_(tau)
        dBdR = dBdR_(tau)
        
        temp = dF*dBdR*dtau
        mask = (tt - tau) < 0
        temp[mask] = 0
        dsdR += temp.sum(axis=-1).unsqueeze(-1)

        dRdw = 1/3*Ro*((ho/hh)**(1/n))*(ww**(-2/3))
        dRdh = -1/n*((ho/hh)**(1/n))*(1/hh)*(ww**(1/3))
        dsdw = dsdR*dRdw
        dsdh = dsdR*dRdh
            
        return s_w, dsdw, s_h, dsdh


    def train(model, loader, optimizer, MaxX, MinX, MaxY, MinY, lam1, lam2):
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model.train()
        total_loss = 0
        
        xmax = torch.from_numpy(MaxX).to(device)
        xmin = torch.from_numpy(MinX).to(device)
        
        for batch_idx, (data, target) in enumerate(loader):
            
            data, target = Variable(data, requires_grad=True).to(device), Variable(target, requires_grad=True).to(device)
            # all_zeros = torch.zeros(target.shape).to(device)
            # print(target.shape)
            
            # normalization
            dataN = (data-xmin)/(xmax-xmin)
                    
            def closure():
                optimizer.zero_grad()
                output = model(torch.reshape(dataN[:,0], (-1,1)), torch.reshape(dataN[:,1], (-1,1)), torch.reshape(dataN[:,2], (-1,1)))
                # print(output.shape)
                mse_d = criterion(output,target)    
                
                s_w, dsdw, s_h, dsdh = f_partial(data, model, MaxX, MinX, MaxY, MinY)
                # print(s_w)
                # print(dsdw)
                
                mse_sw = criterion(s_w,dsdw)
                mse_sh = criterion(s_h,dsdh)
                
                # print(mse_d)
                # print(mse_sw)
                # print(mse_sh)
                
                loss = mse_d + lam1*mse_sw + lam2*mse_sh
                # loss = mse_d
                
                loss.backward()
                return loss
            
            loss = optimizer.step(closure=closure)
            total_loss += loss.item()*len(data)
        
        total_loss /= len(loader.dataset)
        
        return total_loss
        

    def test(model, loader, MaxX, MinX, MaxY, MinY):
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model.eval()
        test_loss = 0
        
        xmax = torch.from_numpy(MaxX).to(device)
        xmin = torch.from_numpy(MinX).to(device)
        
        for data, target in loader:

            data = data.requires_grad_(False).to(device)
            target = target.requires_grad_(False).to(device)
            
            # normalization
            dataN = (data-xmin)/(xmax-xmin)
            
            output = model(torch.reshape(dataN[:,0], (-1,1)), torch.reshape(dataN[:,1], (-1,1)), torch.reshape(dataN[:,2], (-1,1)))
            
            loss = criterion(output,target)
            test_loss += loss.item()*len(data)
            # print(len(data))

        test_loss /= len(loader.dataset)

        return test_loss


    #%% Hyperparameter Selection

    max_epoch = 1000
    max_fail = 8
    batch_size = 4096
    val_ratio = 0.2
    train_ratio = 0.8

    lamda1 = 1e-9 # PDE loss weight
    lamda2 = 1

    train_num = int(numdata*train_ratio)
    val_num = numdata - train_num

    train_dataset = ANN_Dataset(Utrain[:train_num,:], Ztrain[:train_num,:])
    val_dataset = ANN_Dataset(Utrain[train_num:,:], Ztrain[train_num:,:])
    test_dataset = ANN_Dataset(Xtest, Ztest)

    # Data Loader (Input Pipeline)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)


    #%% Training

    # Define Network to Train
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = FCNN_Net(numIn, numOut).to(device)
    # model.double()

    # Select Optimizer and Loss Criterion
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                            weight_decay=0, amsgrad=False)

    fail_count = 0
    min_cost = float('inf')

    timevec = np.zeros((max_epoch, 1))
    for epoch in range(1, max_epoch):
        
        start = timer()    
        train(model, train_loader, optimizer, MaxX, MinX, MaxY, MinY, lamda1, lamda2)    
        RMSE_val = np.sqrt(test(model, val_loader, MaxX, MinX, MaxY, MinY))
        print('\n Epoch: {}  Val set RMSE = {:.8f} \n'.format(epoch, RMSE_val))
            
        if RMSE_val < min_cost:
            min_cost = RMSE_val
            fail_count = 0
        elif fail_count >= max_fail:
            print('Stopping... Maximum number of validation failures exceeded')
            break
        else:
            fail_count += 1
                
        timevec[epoch] = timer() - start
            
    time_avg = timevec[:epoch].mean()
    print('Average time taken for a single epoch is %f seconds.' % time_avg)

    RMSE_test = np.sqrt(test(model, test_loader, MaxX, MinX, MaxY, MinY))
    print('\n Test set RMSE = {:.8f}\n'.format(RMSE_test))



    #%% Validation

    model.to('cpu')

    Xin = (Xtest-MinX)/(MaxX-MinX)
    Xt = torch.from_numpy(Xin)
    Zhat = model(torch.reshape(Xt[:,0], (-1,1)), torch.reshape(Xt[:,1], (-1,1)), torch.reshape(Xt[:,2], (-1,1)))
    Zhat = Zhat.detach().numpy()
    Yhat = Yscaler.inverse_transform(Zhat)

    RMSE_norm = np.sqrt(np.mean((Ytest-Yhat)**2))
    print(RMSE_norm)


    line = np.arange(0, MaxY, 1e-4)
    plt.plot(Ytest[:,0], Yhat[:,0], 'bo')
    plt.plot(line, line, 'k')
    plt.xlabel('true')
    plt.ylabel('predicted')
    plt.title('displacement')
    plt.savefig(path + '/disp_true_v_pred.png')
    # plt.show()
    plt.close('all')
    


    plt.plot(Xtest[:,2], Yhat[:,0], 'bo')
    plt.plot(Xtest[:,2], Ytest[:,0], 'r')
    plt.xlabel('time')
    plt.ylabel('displacement')
    plt.savefig(path + '/disp_err_v_time.png')
    # plt.show()
    plt.close('all')
    



    #%% Save Data and Model

    # v1: lamda1 = 1e-9, lamda2 = 1 (noise 0) RMSE: 0.0001379581


    modelname = 'SeismicWave_TDSF_forward_PGNN_v1.pth'
    ANNmodel = {'model': FCNN_Net(numIn, numOut),
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict()}
    torch.save(ANNmodel, path + '/' + modelname)


    filename = 'SeismicWave_TDSF_forward_PGNN_v1_result.mat'
    sio.savemat(path + '/' + filename, 
                {'Yhat':Yhat, 'Ytest':Ytest, 'Xtest':Xtest,
                'Ymax':MaxY, 'Ymin':MinY, 'Xmax':MaxX, 'Xmin':MinX})


    #%% Result Comparison

    modelname = 'SeismicWave_TDSF_forward_PGNN_v1.pth'
    model_data1 = torch.load(path + '/' + modelname)

    model1 = model_data1['model']
    model1.load_state_dict(model_data1['state_dict'])
    model1.eval()


    TI = 2
    Ind1 = int((TI-1)*100)
    Ind2 = int(TI*100)
    tv = Xtest[Ind1:Ind2,2]
    Yreal = np.squeeze(Ytest[Ind1:Ind2])

    Xin = (Xtest-MinX)/(MaxX-MinX)

    Ztemp1 = model1(torch.from_numpy(np.reshape(Xin[Ind1:Ind2,0],(-1,1))),
                    torch.from_numpy(np.reshape(Xin[Ind1:Ind2,1],(-1,1))),
                    torch.from_numpy(np.reshape(Xin[Ind1:Ind2,2],(-1,1)))).detach().numpy()
    Ytemp1 = np.squeeze(Yscaler.inverse_transform(Ztemp1))




    #%%

    # Plot desired concentration    
    plt.plot(tv, Yreal, 'k')
    plt.plot(tv, Ytemp1, 'b')
    plt.xlabel('time')
    plt.ylabel('displacement')
    plt.legend(('Truth',f'{sigma}% Noise'))
    plt.savefig(path + '/t_v_d.png')
    # plt.show()
    plt.close('all')
    


    # Plot desired concentration    
    plt.plot(tv, np.abs(Ytemp1-Yreal), 'b')
    plt.xlabel('time')
    plt.ylabel('displacement error')
    plt.legend((f'{sigma}% Noise'))
    plt.savefig(path + '/t_v_d_err.png')
    plt.close('all')
    # plt.show()
    
    


    #%% Correlation Check

    from scipy import signal
    import matplotlib.pyplot as plt

    corr1 = signal.correlate(Ytemp1, Yreal, mode='full', method='auto')
    corr1 /= np.max(corr1)
    lags1 = signal.correlation_lags(len(Ytemp1), len(Yreal))

    plt.plot(lags1, corr1, 'b')
    plt.xlabel('lags')
    plt.ylabel('cross-correlation')
    plt.legend((f'{sigma}% Noise'))
    plt.savefig(path + '/cross_corr.png')
    plt.close('all')
    # plt.show()
    


 Epoch: 1  Val set RMSE = 0.12157073 


 Epoch: 2  Val set RMSE = 0.12070863 


 Epoch: 3  Val set RMSE = 0.12062864 


 Epoch: 4  Val set RMSE = 0.12053712 


 Epoch: 5  Val set RMSE = 0.12041357 


 Epoch: 6  Val set RMSE = 0.12024279 


 Epoch: 7  Val set RMSE = 0.12000464 


 Epoch: 8  Val set RMSE = 0.11969151 


 Epoch: 9  Val set RMSE = 0.11932322 


 Epoch: 10  Val set RMSE = 0.11896518 


 Epoch: 11  Val set RMSE = 0.11870596 


 Epoch: 12  Val set RMSE = 0.11853141 


 Epoch: 13  Val set RMSE = 0.11836685 


 Epoch: 14  Val set RMSE = 0.11816579 


 Epoch: 15  Val set RMSE = 0.11789455 


 Epoch: 16  Val set RMSE = 0.11752114 


 Epoch: 17  Val set RMSE = 0.11702134 


 Epoch: 18  Val set RMSE = 0.11633455 


 Epoch: 19  Val set RMSE = 0.11503009 


 Epoch: 20  Val set RMSE = 0.10987388 


 Epoch: 21  Val set RMSE = 0.09279977 


 Epoch: 22  Val set RMSE = 0.07663156 


 Epoch: 23  Val set RMSE = 0.06521630 


 Epoch: 24  Val set RMSE = 0.05627489 


 Epoch: 25  Val set RMSE